In [ ]:
# Basic Libraries
import pandas as pd
import numpy as np

import plotly.figure_factory as ff
from sklearn.model_selection import train_test_split


from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical 


In [ ]:
#Read csv
df = pd.read_csv("Features_CNN.csv")

In [ ]:
X = df.drop(['Class',"slice_file_name"], axis=1)
Y = df['Class']
X = np.array(X)
Y = np.array(Y)

In [ ]:
encoder = LabelEncoder()
Y_encoded = encoder.fit_transform(Y)
Y= to_categorical(Y_encoded)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.2,stratify=Y ,random_state = 0)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 16, 8, 1)
X_test = X_test.reshape(X_test.shape[0], 16, 8, 1)
input_dim = (16, 8, 1)
model = Sequential()

In [ ]:
# Model

model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1024, activation = "tanh"))
model.add(Dropout(0.2))
model.add(Dense(10, activation = "softmax"))

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=5e-4,
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=3,
    min_lr=1e-5
)

In [ ]:
model.fit(X_train, Y_train, epochs = 90, batch_size = 50, validation_data = (X_test, Y_test), callbacks=[early_stopping, reduce_lr])

In [ ]:
# Compute confusion matrix
predicted = np.argmax(model.predict(X_test), axis=1)
true = np.argmax(Y_test, axis=1)
matrix = confusion_matrix(true, predicted)

# Plot confusion matrix
labels = list(encoder.classes_)

fig = ff.create_annotated_heatmap(z=matrix, x=labels, y=labels, colorscale='Viridis')
fig.update_layout(
                  xaxis=dict(title='Predicted Class'),
                  yaxis=dict(title='True Class'),
                  font=dict(size=12),
                  width=1000,
                  height=500)
fig.show()